In [1]:
import keras
import numpy as np
import time
import warnings

from eigenpro import kernels
from eigenpro import mnist
from eigenpro import ciphar
from eigenpro import synthetic
from eigenpro import utils
from eigenpro import training

Using TensorFlow backend.
/home/johannes/seminar/training.py:50: Warning: 

EigenPro-tensorflow has been tested with Keras 2.0.8. If the
current version (2.2.4) fails, switch to 2.0.8 by command,

	pip install Keras==2.0.8


  '\tpip install Keras==2.0.8\n\n' %(keras.__version__), Warning)


In [2]:
### Dataset

dataset_dict = {}

num_classes = 10
(x_train_full, y_train_full), (x_test_full, y_test_full) = mnist.load()
y_train_full = keras.utils.to_categorical(y_train_full, num_classes)
y_test_full = keras.utils.to_categorical(y_test_full, num_classes)
dataset = ((x_train_full, y_train_full), (x_test_full, y_test_full))
dataset_dict['MNIST'] = dataset

# num_classes = 10
# (x_train_full, y_train_full), (x_test_full, y_test_full) = ciphar.load()
# y_train_full = keras.utils.to_categorical(y_train_full, num_classes)
# y_test_full = keras.utils.to_categorical(y_test_full, num_classes)
# dataset = ((x_train_full, y_train_full), (x_test_full, y_test_full))
# dataset_dict['CIPHAR'] = dataset

num_classes = 2
(x_train_full, y_train_full), (x_test_full, y_test_full) = synthetic.load(1)
y_train_full = keras.utils.to_categorical(y_train_full, num_classes)
y_test_full = keras.utils.to_categorical(y_test_full, num_classes)
dataset = ((x_train_full, y_train_full), (x_test_full, y_test_full))
dataset_dict['Synthetic1'] = dataset

# num_classes = 2
# (x_train_full, y_train_full), (x_test_full, y_test_full) = synthetic.load(2)
# y_train_full = keras.utils.to_categorical(y_train_full, num_classes)
# y_test_full = keras.utils.to_categorical(y_test_full, num_classes)
# dataset = ((x_train_full, y_train_full), (x_test_full, y_test_full))
# dataset_dict['Synthetic2'] = dataset

Load MNIST dataset.
60000 train samples
10000 test samples
Generated Synthetic1 dataset.
60000 train samples
10000 test samples


In [3]:
### Kernel

kernel_dict = {}

sg = 5
kernel_sgd = lambda x,y: kernels.Gaussian(x, y, sg)
kernel_inv = lambda x,y: training.Gaussian(x, y, sg)
kernel_dict["Gaussian"] = (kernel_sgd, kernel_inv)

sl = 10
kernel_sgd_l = lambda x,y: kernels.Laplace(x, y, sl)
kernel_inv_l = lambda x,y: training.Laplace(x, y, sl)
kernel_dict["Laplace"] = (kernel_sgd_l, kernel_inv_l)



### Size 

size_list = [400]
# size_list = [10000] # or [60000]



### Noise

noise_list = [0, 100]


### Training

trainers = training.training(dataset_dict, kernel_dict, size_list, noise_list, MAXEPOCH=100)

with open('output/table1-' + time.strftime("%Y%m%d-%H%M%S") + '.txt', 'w') as f:
    print(trainers, file=f)

D:MNIST K:Gaussian S:400 N:0
Instructions for updating:
Colocations handled automatically by placer.
SVD time: 0.20, Eigenvalue ratio: 1975.48
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
indexed-feat- (InputLayer)   (None, 785)               0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 784)               0         
_________________________________________________________________
kernel_embedding_2 (KernelEm (None, 400)               313600    
_________________________________________________________________
trainable (Dense)            (None, 10)                4000      
Total params: 317,600
Trainable params: 4,000
Non-trainable params: 313,600
_________________________________________________________________

Stochastic Gradient Descent
Instructions for updating:
Use tf.cast instead.
train error: 9.50%	test error: 28.55% (1 epo

train error: 0.00%	test error: 49.43% (45 epochs, 3.54 seconds)
Zero Train Error

Linear Interpolation
Classification Error = 0.4934

D:Synthetic1 K:Laplace S:400 N:0
SVD time: 0.14, Eigenvalue ratio: 596.84
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
indexed-feat- (InputLayer)   (None, 51)                0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 50)                0         
_________________________________________________________________
kernel_embedding_2 (KernelEm (None, 400)               20000     
_________________________________________________________________
trainable (Dense)            (None, 2)                 800       
Total params: 20,800
Trainable params: 800
Non-trainable params: 20,000
_________________________________________________________________

Stochastic Gradient Descent
train error: 0.00%	test error: 

In [4]:
# trainers = eval(open('table1.txt', 'r').read())
# trainers

In [24]:
# Table

for name, trainer in trainers.items(): 
    
    print(str(trainer['iterations']) + '\t' + trainer['dataset'] + ' ' + trainer['kernel'] + ' ' 
          +  str(trainer['noise']))

10	MNIST Gaussian 0
23	MNIST Gaussian 100
2	MNIST Laplace 0
3	MNIST Laplace 100
1	Synthetic1 Gaussian 0
45	Synthetic1 Gaussian 100
1	Synthetic1 Laplace 0
4	Synthetic1 Laplace 100


In [8]:
# del trainers

In [25]:
# trainers